In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

In [ ]:
input_df = pd.read_csv('../input/train.csv')
test_df = pd.read_csv('../input/test.csv')
input_df.head()
#input_df.info()

In [ ]:
#2 Second Try
# using dummie - better suited for categorical data

train_data = input_df.copy()

#no garage, equivalent to garage built long time ago (0 AD) - reconsider value
train_data['GarageYrBlt'].fillna(0,inplace = True)
train_data['LotFrontage'].fillna(0, inplace = True)
train_data['MasVnrArea'].fillna(0, inplace = True)

neg_inf = - (1 << 30)
#Basement doesnot exists so -inf (0 is for unfinished basement)
train_data['BsmtFinSF1'].fillna( neg_inf, inplace = True)
train_data['BsmtFinSF2'].fillna( neg_inf, inplace = True)
train_data['BsmtUnfSF'].fillna( neg_inf, inplace = True)
train_data['TotalBsmtSF'].fillna( neg_inf, inplace = True)

train_data['BsmtFullBath'].fillna(0, inplace = True)
train_data['BsmtHalfBath'].fillna(0, inplace = True)

#no garage, no car space (check if correct)
train_data['GarageCars'].fillna(0, inplace = True)
train_data['GarageArea'].fillna(0, inplace = True)

train_data.fillna('NA', inplace = True)
#train_data = train_data.astype('int64')
#train_data.info()

In [ ]:
#df_sex = pd.get_dummies(df['sex'])

colLabels = train_data.columns.values
obj_list=list(item for item in colLabels if train_data[item].dtype==np.object)

train_new = train_data.copy();

for x in obj_list:
    temp = pd.get_dummies(train_data[x], prefix= x + '_')
    train_new = pd.concat([train_new, temp], axis=1)
    train_new.drop([x], axis = 1, inplace = True)
train_new = train_new.astype('int64')
train_new.info()
train_cols = train_new.columns.values

In [ ]:

test_data = test_df.copy()

#no garage, equivalent to garage built long time ago (0 AD) - reconsider value
test_data['GarageYrBlt'].fillna(0,inplace = True)
test_data['LotFrontage'].fillna(0, inplace = True)
test_data['MasVnrArea'].fillna(0, inplace = True)

neg_inf = - (1 << 30)
#Basement doesnot exists so -inf (0 is for unfinished basement)
test_data['BsmtFinSF1'].fillna( neg_inf, inplace = True)
test_data['BsmtFinSF2'].fillna( neg_inf, inplace = True)
test_data['BsmtUnfSF'].fillna( neg_inf, inplace = True)
test_data['TotalBsmtSF'].fillna( neg_inf, inplace = True)

test_data['BsmtFullBath'].fillna(0, inplace = True)
test_data['BsmtHalfBath'].fillna(0, inplace = True)

#no garage, no car space (check if correct)
test_data['GarageCars'].fillna(0, inplace = True)
test_data['GarageArea'].fillna(0, inplace = True)

test_data.fillna('NA', inplace=True)
#train_data = train_data.astype('int64')
#test_data.info()

In [ ]:
colLabels = test_data.columns.values
obj_list=list(item for item in colLabels if test_data[item].dtype==np.object)

test_new = test_data.copy();

for x in obj_list:
    temp = pd.get_dummies(test_data[x], prefix= x + '_')
    test_new = pd.concat([test_new, temp], axis=1)
    test_new.drop([x], axis = 1, inplace = True)
test_new = test_new.astype('int64')
test_new.info()
test_new.head()
test_cols = test_new.columns.values

In [ ]:
diff = []
for x in train_cols:
    if x not in test_cols:
        diff = diff + [x]
print(diff)

diff2 = []
for x in test_cols:
    if x not in train_cols:
        diff2 = diff2 + [x]
print(diff2)
        
#some extra categories available in train data
target = train_new['SalePrice']
train_new_clean = train_new.drop(diff, axis = 1)
train_new_clean.info()

test_new_clean = test_new.drop(diff2, axis = 1) 
train_new_clean.info()

#next improvement try not droping these colums

train_new_clean.drop(['Id'], axis = 1)
test_new_clean.drop(['Id'], axis = 1)

from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(random_state=1, n_estimators=150, min_samples_split=4, min_samples_leaf=2)
rf.fit(train_new_clean, target)
print(rf.score(train_new_clean,target))

pred = rf.predict(test_new_clean)

np.savetxt('rand_forest_dummies.csv', np.c_[range(1461,len(test_new_clean)+1461),pred], delimiter=',', header = 'Id,SalePrice', comments = '', fmt='%d')